# Generate vector tiles

This notebook takes the spatial signature geometries we have saved as a Parquet file and convert them to the vector tiles that can be used in a webmap.

## Clip by coastline

We first clip the signatures by the coastline of GB using the coastline layer we initially used to delimit enclosures.

In [2]:
import os
import geopandas as gpd
from sqlalchemy import create_engine

In [3]:
user = os.environ.get('DB_USER')
pwd = os.environ.get('DB_PWD')
host = os.environ.get('DB_HOST')
port = os.environ.get('DB_PORT')

db_connection_url = f"postgres+psycopg2://{user}:{pwd}@{host}:{port}/built_env"
engine = create_engine(db_connection_url)

In [6]:
sql = f'SELECT * FROM gb_coastline_2016'
coastline = gpd.read_postgis(sql, engine, geom_col='geometry')

In [7]:
from shapely.ops import polygonize

polygons = polygonize(coastline.geometry)

In [11]:
polygons = gpd.GeoSeries([poly for poly in polygons], crs=coastline.crs)
polygons

0       POLYGON ((85970.000 14510.000, 85940.000 14530...
1       POLYGON ((86170.000 14440.000, 86200.000 14430...
2       POLYGON ((86330.000 13970.000, 86350.000 13980...
3       POLYGON ((85950.000 13950.000, 85950.000 13940...
4       POLYGON ((86080.000 15470.000, 86080.000 15450...
                              ...                        
2394    POLYGON ((166426.000 225405.000, 166426.000 22...
2395    POLYGON ((167080.000 225526.000, 167064.000 22...
2396    POLYGON ((171825.000 226293.000, 171839.000 22...
2397    POLYGON ((517572.000 424703.000, 517594.000 42...
2398    POLYGON ((327888.000 562274.000, 327971.000 56...
Length: 2399, dtype: geometry

In [13]:
df = gpd.read_parquet("../../urbangrammar_samba/spatial_signatures/signatures/signatures_combined_levels_orig.pq")

In [14]:
%%time
clipped = gpd.clip(df, polygons)

CPU times: user 26min 54s, sys: 465 ms, total: 26min 55s
Wall time: 26min 55s


In [26]:
# clipped = clipped.explode()
clipped = clipped.reset_index(drop=True)
clipped_polys = clipped[clipped.geom_type=="Polygon"]

In [28]:
clipped_polys.geom_type.value_counts()

Polygon    96936
dtype: int64

## Save to GeoJSON
Clipped and cleaned signature geometry is now ready to be convereted to vector tiles. We use `tippecanoe` to do that. `tippecanoe` is a command-line toolkit that requires GeoJSON in lat/lon as an input.

In [29]:
clipped_polys.reset_index(drop=True)[["signature_type", "geometry"]].to_crs(4326).to_file("../../urbangrammar_samba/spatial_signatures/signatures/signatures_combined_levels_clipped_4326.geojson", driver="GeoJSON")

## Generate tiles
Finally, we run `tippecanoe` and create tiles.

In [30]:
! tippecanoe -z15 \
             --no-tile-compression \
             --output-to-directory=../../great-britain/tiles/ \
             --drop-densest-as-needed \
             --coalesce-smallest-as-needed --extend-zooms-if-still-dropping --detect-shared-borders --coalesce --reorder --hilbert \
             --force \
             ../../urbangrammar_samba/spatial_signatures/signatures/signatures_combined_levels_clipped_4326.geojson

For layer 0, using name "signatures_combined_levels_clipped_4326"
96936 features, 168540234 bytes of geometry, 13651 bytes of separate metadata, 116 bytes of string pool
tile 5/15/10 size is 765139 with detail 12, >500000    
Going to try keeping the sparsest 58.81% of the features to make it fit
tile 5/15/10 size is 731072 with detail 12, >500000    
Going to try keeping the sparsest 36.20% of the features to make it fit
tile 5/15/10 size is 646126 with detail 12, >500000    
Going to try keeping the sparsest 25.21% of the features to make it fit
tile 5/15/10 size is 563230 with detail 12, >500000    
Going to try keeping the sparsest 20.14% of the features to make it fit
tile 6/31/20 size is 812368 with detail 12, >500000    
Going to try keeping the sparsest 55.39% of the features to make it fit
tile 6/31/21 size is 695267 with detail 12, >500000    
Going to try keeping the sparsest 64.72% of the features to make it fit
tile 6/31/20 size is 735976 with detail 12, >500000    
Going 